In [1]:
import openai
import pandas as pd
import numpy as np
from openai.embeddings_utils import get_embedding


COMPLETIONS_MODEL = "gpt-3.5-turbo"
EMBEDDINGS_MODEL = "text-embedding-ada-002"
openai.api_key = ""

In [1]:
# Count the number of tokens if necessary
import tiktoken

def num_tokens_from_string(string: str, encoding_name: str) -> int:
    """Returns the number of tokens in a text string."""
    encoding = tiktoken.get_encoding(encoding_name)
    num_tokens = len(encoding.encode(string))
    return num_tokens


In [ ]:
raw_transcript_df = pd.read_csv("raw_transcript.csv")
raw_transcript_df["sentence"] = raw_transcript_df["output"].str.split('.')
exploded_df = raw_transcript_df.explode("sentence")

In [ ]:
from openai.embeddings_utils import get_embedding

exploded_df = exploded_df[exploded_df['sentence'].str.len() > 0]
exploded_df['embedding'] = exploded_df['sentence'].apply(lambda row: get_embedding(row, engine=EMBEDDINGS_MODEL))
exploded_df.to_csv('embeddings_transcript.csv')

In [ ]:
#k-means clustering
import numpy as np
from sklearn.cluster import KMeans

exploded_parent_df = pd.read_csv("embeddings_transcript.csv")
clustered_text_df = pd.DataFrame(columns=['url', 'title', 'cluster', 'aggregated_text'])

for item in exploded_parent_df['url'].unique():
    embedding_df = exploded_parent_df.loc[exploded_parent_df['url'] == item].copy()
    embedding_df["embedding"] = embedding_df.embedding.apply(eval).apply(np.array)  # convert string to numpy array
    matrix = np.vstack(embedding_df.embedding.values)

    n_clusters = 2 #arbitrary

    kmeans = KMeans(n_clusters=n_clusters, init="k-means++", random_state=42)
    kmeans.fit(matrix)
    labels = kmeans.labels_
    embedding_df["cluster"] = labels

    combined_df = embedding_df.groupby(['url', 'title', 'cluster'])['sentence'].apply('. '.join).reset_index()
    combined_df['aggregated_text'] = combined_df['title'] + ', ' + combined_df["sentence"]
    combined_df = combined_df.drop(['sentence'], axis=1)
    clustered_text_df = pd.concat([clustered_text_df,combined_df])

clustered_text_df.to_csv('clustered_text.csv')

In [ ]:
clustered_text_df = pd.read_csv("clustered_text.csv")

clustered_text_df['embedding'] = clustered_text_df['aggregated_text'].apply(lambda row: get_embedding(row, engine=EMBEDDINGS_MODEL))
clustered_text_df.to_csv('clustered_embeddings.csv')

In [19]:
from openai.embeddings_utils import cosine_similarity

clustered_embeddings_df = pd.read_csv('clustered_embeddings.csv')
clustered_embeddings_df['embedding'] = clustered_embeddings_df['embedding'].apply(eval).apply(np.array)

question1 = "What is the law of demand?"
question1_vector = get_embedding(question1, engine=EMBEDDINGS_MODEL)

question2 = "What is demand elasticity?"
question2_vector = get_embedding(question2, engine=EMBEDDINGS_MODEL)

clustered_embeddings_df["similarities"] = clustered_embeddings_df['embedding'].apply(lambda x: cosine_similarity(x, question2_vector))
sorted_embeddings = clustered_embeddings_df.sort_values("similarities", ascending=False).head(2)

sorted_embeddings

,Unnamed: 0.1,Unnamed: 0,url,title,cluster,aggregated_text,embedding,similarities
2,2,0,https://www.youtube.com/watch?v=slP8XZ6Nq40,Price Elasticity of Demand using the midpoint ...,0,Price Elasticity of Demand using the midpoint ...,"[-0.004820945672690868, -0.002884791698306799,...",0.873718
4,4,0,https://www.youtube.com/watch?v=6udRtn5jSWk,Perfect inelasticity and perfect elasticity of...,0,Perfect inelasticity and perfect elasticity of...,"[-0.002379822777584195, -0.013577613048255444,...",0.859431


In [20]:
context = []
for i, row in sorted_embeddings.iterrows():
  context.append(row['aggregated_text'][:1300])  # limit the number of tokens per matched sequence to 1300 tokens

text = "\n".join(context)
context = text
text

"Price Elasticity of Demand using the midpoint method Microeconomic, What we're going to think about in this video is elasticity of demand.  Elasticity of demand.  And what this is, is a measure of how does the quantity demanded change given a change in price, or how does a change in price impact the quantity demanded.  So change in price impact quantity demanded.  When you talk about demand, you're talking about the whole curve.  Quantity demanded is a specific quantity.  Quantity demanded.  The way that economists measure this is they measure it as a percent change in quantity over the percent change in price.  And the reason why they do this, as opposed to just say change in quantity over change in price, is because if you did change in quantity over change in price, you would have a number that's specific to the units you're using.  So it would depend on whether you're doing quantity in terms of per hour, per week, or per year.  Because of the percentage, you're taking a change in 

In [21]:
from langchain import FewShotPromptTemplate
from langchain import PromptTemplate

# create our examples
examples = [
    {
        "query": "How are you?",
        "answer": "I can't complain but sometimes I still do."
    }, {
        "query": "What is the law of demand?",
        "answer": "The law of demand is that the higher the price, the lower the quantity demanded, and the lower the price, the higher the quantity demanded. It's like the law of gravity, but for economics."
    }
]

# create a example template
example_template = """
User: {query}
AI: {answer}
"""

# create a prompt example from above template
example_prompt = PromptTemplate(
    input_variables=["query", "answer"],
    template=example_template
)

# now break our previous prompt into a prefix and suffix
# the prefix is our instructions
prefix = """The following are exerpts from conversations with an AI
economics tutor. The tutor is typically sarcastic and witty, producing
creative  and funny responses to the users questions. The tutor responds
using the context provided. 

Context: {context}

Here are some response examples: 
"""
# and the suffix our user input and output indicator
suffix = """
User: {query}
AI: """

# now create the few shot prompt template
few_shot_prompt_template = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    prefix=prefix,
    suffix=suffix,
    input_variables=["query", "context"],
    example_separator="\n\n"
)

In [22]:
user_prompt = f"""{few_shot_prompt_template.format(query=question2, context=context)}"""

print(user_prompt)


The following are exerpts from conversations with an AI
economics tutor. The tutor is typically sarcastic and witty, producing
creative  and funny responses to the users questions. The tutor responds
using the context provided. 

Context: Price Elasticity of Demand using the midpoint method Microeconomic, What we're going to think about in this video is elasticity of demand.  Elasticity of demand.  And what this is, is a measure of how does the quantity demanded change given a change in price, or how does a change in price impact the quantity demanded.  So change in price impact quantity demanded.  When you talk about demand, you're talking about the whole curve.  Quantity demanded is a specific quantity.  Quantity demanded.  The way that economists measure this is they measure it as a percent change in quantity over the percent change in price.  And the reason why they do this, as opposed to just say change in quantity over change in price, is because if you did change in quantity ove

In [27]:
system_prompt = f""""""

openai.ChatCompletion.create(
    temperature=0.3,
    max_tokens=700,
    top_p=1,
    frequency_penalty=0,
    presence_penalty=0,
    model=COMPLETIONS_MODEL,
    messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt}
    ]
)["choices"][0]["message"]

<OpenAIObject at 0x25e82b60400> JSON: {
  "content": "Demand elasticity is a measure of how responsive the quantity demanded is to changes in price. It's like a relationship status on Facebook - it's complicated.",
  "role": "assistant"
}